In [1]:
import pynq
import time
from PIL import Image
import numpy as np

# Open BMP file
img = Image.open("xilinx_img.bmp")
goldImg = Image.open("golden.bmp")

#print(img.size)

# Convert image to NumPy array
img_arr = np.array(img)
golden_arr = np.array(goldImg)
#print(img_arr.shape)

# Convert each pixel from (R, G, B) to a single 4-byte integer
pixel_arr = np.zeros((img_arr.shape[0] * img_arr.shape[1]), dtype=np.uint32)
golden_pixel_arr = np.zeros((golden_arr.shape[0] * golden_arr.shape[1]), dtype=np.uint32)
for i in range(img_arr.shape[0]):
    for j in range(img_arr.shape[1]):
        pixel_arr[(i * img_arr.shape[1] + j)] = pixel_arr[(i * img_arr.shape[1] + j)] | img_arr[i, j, 0]
        pixel_arr[(i * img_arr.shape[1] + j)] = pixel_arr[(i * img_arr.shape[1] + j)] |(img_arr[i, j, 1] << 8)
        pixel_arr[(i * img_arr.shape[1] + j)] = pixel_arr[(i * img_arr.shape[1] + j)] |(img_arr[i, j, 2] << 16)
        golden_pixel_arr[(i * img_arr.shape[1] + j)] = golden_pixel_arr[(i * img_arr.shape[1] + j)] | golden_arr[i, j, 0]
        golden_pixel_arr[(i * img_arr.shape[1] + j)] = golden_pixel_arr[(i * img_arr.shape[1] + j)] |(golden_arr[i, j, 1] << 8)
        golden_pixel_arr[(i * img_arr.shape[1] + j)] = golden_pixel_arr[(i * img_arr.shape[1] + j)] |(golden_arr[i, j, 2] << 16)

In [2]:
bitstream = "applyWatermark_avoid_critical_path.bit"
overlay = pynq.Overlay(bitstream)

#overlay?




In [3]:
dut = overlay.apply_watermark_0
#dut?




In [4]:
fpgaPixel = pynq.allocate((img_arr.shape[0] * img_arr.shape[1]), dtype='int32')
fpgaOutput= pynq.allocate((img_arr.shape[0] * img_arr.shape[1]), dtype='int32')

fpgaPixel[:] = pixel_arr

fpgaPixel.sync_to_device()

#dut.register_map

In [5]:
dut.register_map.input_r_1 = fpgaPixel.device_address & 0xFFFFFFFF
dut.register_map.input_r_2 = fpgaPixel.device_address >> 32

dut.register_map.output_r_1 = fpgaOutput.device_address & 0xFFFFFFFF
dut.register_map.output_r_2 = fpgaOutput.device_address >> 32

start_time = time.time()

dut.register_map.CTRL.AP_START = 1
dut.register_map.CTRL[4] = 1
while not dut.register_map.CTRL.AP_DONE: pass

end_time = time.time()
duration = end_time - start_time
print(f'Kernel completed in {duration * 1000:.2f}ms')

Kernel completed in 5.48ms


In [6]:
fpgaOutput.sync_from_device()
output_pixel_arr = np.zeros((golden_arr.shape[0] * golden_arr.shape[1]), dtype=np.uint32)
output_pixel_arr[:] = fpgaOutput

In [7]:
# Your code goes here (verify)
match = 1
numOfPix = golden_arr.shape[0] * golden_arr.shape[1]
for i in range(numOfPix):
    if (output_pixel_arr[i] != golden_pixel_arr[i]):
        match = 0
        print("pixel mismatch at",i," location")
        break
if match:
    print("!!!!!!!!Execution SUCCESS!!!!!!!!!!!!")
else:
    print("!!!!!!!!Execution FAILED!!!!!!!!!!!!")
overlay.free()

!!!!!!!!Execution SUCCESS!!!!!!!!!!!!


In [8]:
img_arr_new = np.empty((img_arr.shape[0], img_arr.shape[1], img_arr.shape[2]), dtype=np.uint8)
for i in range(img_arr.shape[0]):
    for j in range(img_arr.shape[1]):
        img_arr_new[i, j, 0] = (output_pixel_arr[(i * img_arr.shape[1] + j)])& 0xff
        img_arr_new[i, j, 1] = (output_pixel_arr[(i * img_arr.shape[1] + j)] >> 8)& 0xff
        img_arr_new[i, j, 2] = (output_pixel_arr[(i * img_arr.shape[1] + j)] >> 16)& 0xff
        
new_img = Image.fromarray(img_arr_new, mode="RGB")
new_img.save("new_file_name32.bmp")